# Top 10 Position with First Genetic Algorithm on GPU! 🔥


## How I came up with this solution?

For the Kaggle [Conway's Reverse Game of Life 2020](https://www.kaggle.com/c/conways-reverse-game-of-life-2020/overview) competition, I tried many approaches and I also wanted to try something new for me: **Genetic Algorithm**. 

I looked up existing Kaggle notebooks using Genetic Algorithm, but none of them were using a GPU accelerator (sorry if I am wrong and missed one). It puzzled me as the GPU is faster than the CPU. I told myself "challenge accepted, let's make a genetic algorithm on the GPU!".

Implementing this genetic algorithm from scratch was tought but I learned a lot in the process about genetic algorithm (first purpose), but also on `pytorch` and how to optimize using `NVIDIA Nsight`.


## Results

This solution performed very well during this competition as this present notebook alone [scores in the top-10 leaderboard](https://www.kaggle.com/c/conways-reverse-game-of-life-2020/leaderboard). This new approach helped improving final solution of our team `Under a Penny`.

I noticed the algorithm is running faster on RTX-2080 than Kaggle's P-100, probably thanks to the newer version of tensor-cores.


## Requirements

All we need are :

 - `pytorch` to implement the Genetic Algorithm on GPU
 - `pandas` to load and write .csv files

## Implementation

In [1]:
import time
import pandas as pd
import torch

In [2]:
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False

### Constants

In [3]:
N = 25  # grid dimension
device = 'cuda'
TEST_CSV = '../input/conways-reverse-game-of-life-2020/test.csv'
OUTPUT_CSV = 'submission.csv'

### Apply Game Of Life steps

`forward` : applies Game Of Life rules `delta` times on a board of cells (`grid`).

The rules are simple and are defined in the [Kaggle competition description](https://www.kaggle.com/c/conways-reverse-game-of-life-2020/overview/description):

> The Game of Life is a cellular automaton created by mathematician John Conway in 1970. The game consists of a board of cells that are either on or off. One creates an initial configuration of these on/off states and observes how it evolves. There are four simple rules to determine the next state of the game board, given the current state:
>
>    - *Overpopulation*: if a living cell is surrounded by more than three living cells, it dies.
>    - *Stasis*: if a living cell is surrounded by two or three living cells, it survives.
>    - *Underpopulation*: if a living cell is surrounded by fewer than two living cells, it dies.
>    - *Reproduction*: if a dead cell is surrounded by exactly three cells, it becomes a live cell.


In [4]:
cv = torch.nn.Conv2d(1, 1, kernel_size=3, padding=1, padding_mode='circular', bias=False)
cv.requires_grad=False
cv.weight = torch.nn.Parameter(
    torch.tensor(
        [[[[ 1., 1., 1.],
           [ 1., 0., 1.],
           [ 1., 1., 1.]]]],
        device=device,
        dtype=torch.float16
    ),
    requires_grad=False,
)


@torch.jit.script
def forward(grid, delta: int):
    N=25
    g = grid.reshape(-1, 1, N, N)
    for _ in torch.arange(delta):
        g = g.to(torch.float16)
        neighbor_sum = cv(g)
        g = ((neighbor_sum == 3) | ((g == 1) & (neighbor_sum == 2)))
    return g.reshape(-1, N, N)

### Genetic algorithm

I highly suggest to read [How the Genetic Algorithm works](https://www.mathworks.com/help/gads/how-the-genetic-algorithm-works.html) webpage as it explains many concepts mentionned in following sections of this notebook.

`random_parents` : create a random initial population made of `n_parents` NxN grids

In [5]:
@torch.jit.script
def random_parents(n_parents: int, device: str):
    N = 25
    RANDOM_ALIVE = .2
    return torch.rand((n_parents, N, N), device=device) > (1-RANDOM_ALIVE)

`select_best`: Scores each member of the current population `parents` by computing its fitness value. For this competition, the fitness value is the number of errors between a `target` and a population individual after applying `delta` time the Game Of Live rules. The functions returns the `n_best` best performing individuals.

In [6]:
@torch.jit.script
def loss(input, target):
    return torch.sum(input ^ target, dim=(-1,-2))

In [7]:
@torch.jit.script
def select_best(parents, delta: int, target, n_best: int):
    scores = loss(forward(parents, delta), target)
    best_values, best_indices = torch.topk(scores, n_best, dim=0, largest=False, sorted=True)
    new_parents = parents[best_indices, ...]
    return new_parents, best_values[0], new_parents[0, ...]

`random_combine`: combining the vector entries of a pair of parents. It is also called _crossover_.

In [8]:
@torch.jit.script
def random_combine(parents, n_offsprings: int, device: str, pre_masks):
    N = 25
    
    dads = torch.randint(low=0, high=parents.shape[0], size=(n_offsprings,),
                         device=device, dtype=torch.long)
    dads = parents[dads, ...]
    
    moms = torch.randint(low=0, high=parents.shape[0], size=(n_offsprings,),
                         device=device, dtype=torch.long)
    moms = parents[moms, ...]
    
    masks = pre_masks[torch.randint(low=0, high=pre_masks.shape[0], size=(n_offsprings,),
                                    device=device, dtype=torch.long)]

    return torch.where(masks, dads, moms)

`precomputes_masks`: pre-computes masked used for combining the vector entries of a pair of parents in `random_combine`.

In [9]:
def precomputes_masks():
    N = 25
    BLOCK_SIZE = 17

    block = torch.nn.Conv2d(1, 1, kernel_size=BLOCK_SIZE, padding=BLOCK_SIZE//2,
                            padding_mode='circular', bias=False)
    block.requires_grad=False
    block.weight = torch.nn.Parameter(
        torch.ones((1, 1, BLOCK_SIZE, BLOCK_SIZE),
            device=device,
            dtype=torch.float16
        ),
        requires_grad=False,
    )

    masks = torch.zeros((N * N, 1, N, N), device=device, dtype=torch.float16)
    
    for x in range(N):
        for y in range(N):
            masks[x * N + y, 0, x, y] = 1.
    masks = block(masks)
    
    return masks[:, 0, ...] > .5

`mutate`: makes random changes on parents.

In [10]:
@torch.jit.script
def mutate(parents, device: str):
    MUTATION = .0016  # .005 
    mutations = torch.rand(parents.shape, device=device) < MUTATION
    return parents ^ mutations

`optimize_one_puzzle`: runs our genetic algorithm on one puzzle

In [11]:
@torch.jit.script
def optimize_one_puzzle(delta: int, data, device: str, pre_masks):
    N = 25
    N_GENERATION = 30  # Number of generations
    P = 4_500  # population
    N_BEST = P // 30  # best to keep as new parents
    N_ELITES = 8  # parents unchanged for next generation
    
    best_score = torch.tensor([N*N], device=device)
    best = torch.zeros((N,N), device=device).to(torch.bool)
    parents = random_parents(P, device)

    elites = torch.empty((1, N, N), dtype=torch.bool, device=device)
    elites[0, ...] = data  # set target as potential dad ;)

    for i in range(N_GENERATION):
        parents = random_combine(parents, P, device, pre_masks)
        parents = mutate(parents, device)
        parents[:N_ELITES, ...] = elites
        parents, best_score, best = select_best(parents, delta, data, N_BEST)
        # Some of the individuals in the current population that have lower fitness are chosen as elite.
        # These elite individuals are passed to the next population.
        elites = parents[:N_ELITES, ...]
        if best_score == 0:  # early stopping
            break

    return best_score, best

`optimize_all_puzzles`: It tries to find approximate solution for all puzzles 😃

In [12]:
@torch.jit.script
def optimize_all_puzzles(deltas, df, device: str, pre_masks):
    sub = df.clone()
    
    for n in torch.arange(df.shape[0]):
        delta = deltas[n]
        data = df[n, ...]
        _, sub[n, ...] = optimize_one_puzzle(delta, data, device, pre_masks)

    return sub

### Load Puzzles to solve

In [13]:
df = pd.read_csv(TEST_CSV, index_col='id')

### Use final state as our baseline for submission

In [14]:
submission = df.copy()
submission.drop(['delta'], inplace=True, axis=1)

### Push data to GPU

In [15]:
indexes = df.index
deltas = torch.from_numpy(df.delta.values).to(device)
df = torch.BoolTensor(df.values[:, 1:].reshape((-1, N, N))).to(device)

### Let's optimize puzzles!

In [16]:
start_time = time.time()
pre_masks = precomputes_masks()
sub = optimize_all_puzzles(deltas, df, device, pre_masks)
print(f'Processed {sub.shape[0]:,} puzzles in {time.time() - start_time:.2f} seconds 🔥🔥🔥')

Processed 50,000 puzzles in 7127.60 seconds 🔥🔥🔥


### Save our submission

In [17]:
submission.rename(columns={f'stop_{x}': f'start_{x}' for x in range(N*N)}, inplace=True)
submission.iloc[:sub.shape[0], :] = sub.reshape((-1, N*N)).cpu().numpy().astype(int)
submission.to_csv(OUTPUT_CSV)

### Compute leaderboard score

In [18]:
def leaderboard_score(deltas, df, sub, device: str):
    result = torch.empty(sub.shape[0], device=device, dtype=torch.long)
    for delta in range(1, 6):
        start = sub[deltas == delta]
        end   = df[deltas == delta]
        result[deltas == delta] = loss(forward(start, delta), end)
    print('Leaderboard score:', torch.sum(result).item() / (result.shape[0]*N*N))

In [19]:
leaderboard_score(deltas, df, sub, device)

Leaderboard score: 0.054160928


# Conclusion

I learned a lot along the way about Genetic Algorithm, pytorch and how to optimize computation on the GPU.

If you like this notebook, please leave a comment 🖊, upvote 👍, and put a smile on your face 😀.